In [2]:
import pandas as pd
import sqlite3

In [3]:
conn=sqlite3.connect("retail.db")

In [4]:
tables=pd.read_sql("select name from sqlite_master where type='table'",conn)
tables

,name
0,Customers
1,Data_Dictionary
2,Exchange_Rates
3,Products
4,Sales
5,Stores


In [5]:
for table in tables['name']:
    print("-"*50,table,"-"*50)
    print(f'No. of records :',pd.read_sql(f'select count(*) as count from {table}',conn)['count'][0])
    display(pd.read_sql_query(f'select * from {table} limit 5',conn))

-------------------------------------------------- Customers --------------------------------------------------
No. of records : 15266


,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965


-------------------------------------------------- Data_Dictionary --------------------------------------------------
No. of records : 37


,Table,Field,Description
0,Sales,Order Number,Unique ID for each order
1,Sales,Line Item,Identifies individual products purchased as pa...
2,Sales,Order Date,Date the order was placed
3,Sales,Delivery Date,Date the order was delivered
4,Sales,CustomerKey,Unique key identifying which customer placed t...


-------------------------------------------------- Exchange_Rates --------------------------------------------------
No. of records : 11215


...

,ProductKey,Product_Name,Brand,Color,Unit_Cost_USD,Unit_Price_USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,7.89,14.57,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,5.20,12.36,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,5.47,10.40,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,7.89,12.63,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,5.55,12.64,101,MP4&MP3,1,Audio


-------------------------------------------------- Sales --------------------------------------------------
No. of records : 62884


,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,01-01-2016,None,265598,10,1304,1.0,CAD
1,366001,1,01-01-2016,1/13/2016,1269051,0,1048,2.0,USD
2,366001,2,01-01-2016,1/13/2016,1269051,0,2007,1.0,USD
3,366002,1,01-01-2016,01-12-2016,266019,0,1106,7.0,CAD
4,366002,2,01-01-2016,01-12-2016,266019,0,373,1.0,CAD


-------------------------------------------------- Stores --------------------------------------------------
No. of records : 67


,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015


In [6]:
sales=pd.read_sql('select * from sales where customerkey=266019',conn)
sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366002,1,01-01-2016,01-12-2016,266019,0,1106,7.0,CAD
1,366002,2,01-01-2016,01-12-2016,266019,0,373,1.0,CAD
2,366002,3,01-01-2016,01-12-2016,266019,0,1080,4.0,CAD
3,1469029,1,01-08-2019,None,266019,10,1453,1.0,CAD
4,1469029,2,01-08-2019,None,266019,10,1738,8.0,CAD
5,1469029,3,01-08-2019,None,266019,10,1443,4.0,CAD


In [7]:
customers=pd.read_sql('select * from customers where customerkey=266019',conn)
customers

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,266019,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,8/24/1962


In [8]:
products=pd.read_sql_query('select sum(unit_cost_usd),sum(unit_price_usd),category from products group by category',conn)
products

,sum(unit_cost_usd),sum(unit_price_usd),Category
0,867.07,1433.25,Audio
1,2791.78,4597.96,Cameras and camcorders
2,2152.16,3484.49,Cell phones
3,4531.27,7622.07,Computers
4,1242.81,2097.13,Games and Toys
5,4944.99,8301.66,Home Appliances
6,639.26,1133.67,"Music, Movies and Audio Books"
7,1666.41,2799.78,TV and Video


In [9]:
customers.columns

Index(['CustomerKey', 'Gender', 'Name', 'City', 'State Code', 'State',
       'Zip Code', 'Country', 'Continent', 'Birthday'],
      dtype='object')

In [10]:
Sales=pd.read_sql_query(""" select 
c.customerkey,
c.name,
c.city,
c.state,
c.country,
sum(s.quantity) as quantity
from customers c
left join
sales s
on c.customerkey=s.customerkey
group by c.customerkey,c.name,
c.city,
c.state,
c.country
""",conn)


In [11]:
Sales.head()

,CustomerKey,Name,City,State,Country,quantity
0,301,Lilly Harding,WANDEARAH EAST,South Australia,Australia,2.0
1,325,Madison Hull,MOUNT BUDD,Western Australia,Australia,27.0
2,554,Claire Ferres,WINJALLOK,Victoria,Australia,14.0
3,786,Jai Poltpalingada,MIDDLE RIVER,South Australia,Australia,NaN
4,1042,Aidan Pankhurst,TAWONGA SOUTH,Victoria,Australia,6.0


In [12]:
Products=pd.read_sql_query("""
select productkey,
product_name,
category,
sum(unit_cost_usd) as Unit_Cost_USD,
sum(unit_price_usd) as Unit_Price_USD
from products
group by productkey,
product_name,
category

""",conn)

In [13]:
Products.head()

,ProductKey,Product_Name,Category,Unit_Cost_USD,Unit_Price_USD
0,1,Contoso 512MB MP3 Player E51 Silver,Audio,7.89,14.57
1,2,Contoso 512MB MP3 Player E51 Blue,Audio,5.20,12.36
2,3,Contoso 1G MP3 Player E100 White,Audio,5.47,10.40
3,4,Contoso 2G MP3 Player E200 Silver,Audio,7.89,12.63
4,5,Contoso 2G MP3 Player E200 Red,Audio,5.55,12.64


In [14]:
Stores=pd.read_sql_query("""
select storekey,
state as Store_State,
country as Store_Country
from stores

""",conn)

In [15]:
Stores.head()

,StoreKey,Store_State,Store_Country
0,1,Australian Capital Territory,Australia
1,2,Northern Territory,Australia
2,3,South Australia,Australia
3,4,Tasmania,Australia
4,5,Victoria,Australia


In [16]:
Summary_Table=pd.read_sql_query("""
with Customers_Summary as(
select 
c.customerkey,
s.productkey,
s.storekey,
c.name,
c.city,
c.state,
c.country,
sum(s.quantity) as Total_Quantity
from customers c
left join
sales s
on c.customerkey=s.customerkey
group by c.customerkey,c.name,s.productkey,s.storekey,
c.city,
c.state,
c.country
),

Products_Summary as(

select productkey,
product_name,
category,
sum(unit_cost_usd) as Unit_Cost_USD,
sum(unit_price_usd) as Unit_Price_USD
from products
group by productkey,
product_name,
category
),

Stores_Summary as(
select storekey,
state as Store_State,
country as Store_Country
from stores
)

select 
cs.Customerkey as Customer_ID,
cs.name as Customer_Name,
cs.city,
cs.state,
cs.country,
ps.product_name,
ps.category,
ss.storekey,
ss.Store_State,
ss.Store_Country,
cs.Total_Quantity,
ps.Unit_Cost_USD,
ps.Unit_Price_USD

from Customers_Summary as cs
join Products_Summary as ps
on cs.productkey=ps.productkey

join Stores_Summary as ss
on cs.storekey=ss.storekey

order by ps.Unit_Price_USD


""",conn)

In [17]:
Summary_Table.head()

,Customer_ID,Customer_Name,city,state,country,product_name,category,storekey,Store_State,Store_Country,Total_Quantity,Unit_Cost_USD,Unit_Price_USD
0,117386,Christopher Currie,WAMBERAL,New South Wales,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,2.0,9.06,10.0
1,122096,Alannah Wolinski,PAMPAS,Queensland,Australia,WWI Laptop8.9 E0089 Black,Computers,6,Western Australia,Australia,4.0,5.40,10.0
2,125236,Jordan Sissons,MORNINGTON ISLAND,Queensland,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,1.0,9.06,10.0
3,127417,Elizabeth Brookman,RIDDELLS CREEK,Victoria,Australia,WWI Laptop8.9 E0089 Black,Computers,5,Victoria,Australia,7.0,5.40,10.0
4,211809,Alexander Norman,Windsor,Ontario,Canada,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,10,Nunavut,Canada,1.0,9.06,10.0


In [18]:
Summary_Table.isnull().sum()

Customer_ID       0
Customer_Name     0
city              0
state             0
country           0
product_name      0
category          0
storekey          0
Store_State       0
Store_Country     0
Total_Quantity    0
Unit_Cost_USD     0
Unit_Price_USD    0
dtype: int64

In [19]:
Summary_Table['Customer_ID'].nunique()

11887

In [20]:
Summary_Table['Total_Cost']=Summary_Table['Unit_Cost_USD']*Summary_Table['Total_Quantity']

In [21]:
Summary_Table['Total_Sales']=Summary_Table['Unit_Price_USD']*Summary_Table['Total_Quantity']

In [22]:
Summary_Table.head()

,Customer_ID,Customer_Name,city,state,country,product_name,category,storekey,Store_State,Store_Country,Total_Quantity,Unit_Cost_USD,Unit_Price_USD,Total_Cost,Total_Sales
0,117386,Christopher Currie,WAMBERAL,New South Wales,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,2.0,9.06,10.0,18.12,20.0
1,122096,Alannah Wolinski,PAMPAS,Queensland,Australia,WWI Laptop8.9 E0089 Black,Computers,6,Western Australia,Australia,4.0,5.40,10.0,21.60,40.0
2,125236,Jordan Sissons,MORNINGTON ISLAND,Queensland,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,1.0,9.06,10.0,9.06,10.0
3,127417,Elizabeth Brookman,RIDDELLS CREEK,Victoria,Australia,WWI Laptop8.9 E0089 Black,Computers,5,Victoria,Australia,7.0,5.40,10.0,37.80,70.0
4,211809,Alexander Norman,Windsor,Ontario,Canada,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,10,Nunavut,Canada,1.0,9.06,10.0,9.06,10.0


In [23]:
Summary_Table['Profit_Margin']=Summary_Table['Total_Sales']-Summary_Table['Total_Cost']

In [24]:
Summary_Table['Gross_Margin']=round(Summary_Table['Profit_Margin']/Summary_Table['Total_Sales'],2)

In [26]:
Summary_Table.head()

,Customer_ID,Customer_Name,city,state,country,product_name,category,storekey,Store_State,Store_Country,Total_Quantity,Unit_Cost_USD,Unit_Price_USD,Total_Cost,Total_Sales,Profit_Margin,Gross_Margin
0,117386,Christopher Currie,WAMBERAL,New South Wales,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,2.0,9.06,10.0,18.12,20.0,1.88,0.09
1,122096,Alannah Wolinski,PAMPAS,Queensland,Australia,WWI Laptop8.9 E0089 Black,Computers,6,Western Australia,Australia,4.0,5.40,10.0,21.60,40.0,18.40,0.46
2,125236,Jordan Sissons,MORNINGTON ISLAND,Queensland,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,1.0,9.06,10.0,9.06,10.0,0.94,0.09
3,127417,Elizabeth Brookman,RIDDELLS CREEK,Victoria,Australia,WWI Laptop8.9 E0089 Black,Computers,5,Victoria,Australia,7.0,5.40,10.0,37.80,70.0,32.20,0.46
4,211809,Alexander Norman,Windsor,Ontario,Canada,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,10,Nunavut,Canada,1.0,9.06,10.0,9.06,10.0,0.94,0.09


In [41]:
Summary_Table.rename(columns={'Profit_Margin': 'Gross_Profit'}, inplace=True) 

In [42]:
 Summary_Table.head()

,Customer_ID,Customer_Name,city,state,country,product_name,category,storekey,Store_State,Store_Country,Total_Quantity,Unit_Cost_USD,Unit_Price_USD,Total_Cost,Total_Sales,Gross_Profit,Gross_Margin
0,117386,Christopher Currie,WAMBERAL,New South Wales,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,2.0,9.06,10.0,18.12,20.0,1.88,0.09
1,122096,Alannah Wolinski,PAMPAS,Queensland,Australia,WWI Laptop8.9 E0089 Black,Computers,6,Western Australia,Australia,4.0,5.40,10.0,21.60,40.0,18.40,0.46
2,125236,Jordan Sissons,MORNINGTON ISLAND,Queensland,Australia,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,1,Australian Capital Territory,Australia,1.0,9.06,10.0,9.06,10.0,0.94,0.09
3,127417,Elizabeth Brookman,RIDDELLS CREEK,Victoria,Australia,WWI Laptop8.9 E0089 Black,Computers,5,Victoria,Australia,7.0,5.40,10.0,37.80,70.0,32.20,0.46
4,211809,Alexander Norman,Windsor,Ontario,Canada,NT Wireless Bluetooth Stereo Headphones M402 S...,Audio,10,Nunavut,Canada,1.0,9.06,10.0,9.06,10.0,0.94,0.09
